In [1]:
'''
To compute CD signatures for LINCS L1000 level3 data using 
all profiles in the same batch as controls
and add results to mongodb.
''' 
import os, sys, json
import h5py
from pymongo import MongoClient
import geode

def file2list(fn, idx, sep='\t', header=False):
	"""read a file into a list"""
	l = []
	with open (fn, 'r') as f:
		if header:
			next(f)
		for line in f:
			if not line.startswith('#'):
				sl = line.strip().split(sep)
				t = sl[idx]
				l.append(t)
	return l


In [2]:
import numpy as np
import pandas as pd

# 0. Load LINCS L1000 level 3 data from gctx file and made function to subset this matrix by probe_ids and sample_ids

In [3]:
# GCTX_FILE = '../../q2norm_n1328098x22268.gctx'
# PROBES = json.load(open('../data/rid.json', 'rb'))
# PROBES_LM1000 = file2list('../data/rid_lm1000.txt', 0)

GCTX_FILE = '/Users/zichen/Documents/Zichen_Projects/L1000_DR/data/q2norm_n1328098x22268.gctx'
PROBES = json.load(open('/Users/zichen/Documents/bitbucket/lincs_l1000_limma/rid.json', 'rb'))
PROBES_LM1000 = file2list('/Users/zichen/Documents/bitbucket/lincs_l1000_limma/rid_lm1000.txt', 0)

print len(PROBES), len(PROBES_LM1000)
print PROBES[:5]
print PROBES_LM1000[:5]

22268 978
[u'200814_at', u'222103_at', u'201453_x_at', u'204131_s_at', u'200059_s_at']
['200814_at', '222103_at', '201453_x_at', '204131_s_at', '200059_s_at']


In [4]:
gctx = h5py.File(GCTX_FILE, 'r')
mat = gctx['/0/DATA/0/matrix']
print mat.shape

gene_ids = list(gctx['/0/META/ROW/id'])
print len(gene_ids), gene_ids[:5]

(1328098, 22268)
22268 ['200814_at', '222103_at', '201453_x_at', '204131_s_at', '200059_s_at']


In [5]:
gctx['/0/META/COL/id'][:5]

array(['CPC005_A375_6H_X1_B3_DUO52HI53LO:K06',
       'CPC005_A375_6H_X2_B3_DUO52HI53LO:K06',
       'CPC005_A375_6H_X3_B3_DUO52HI53LO:K06',
       'CPC005_A375_6H_X1_B3_DUO52HI53LO:C19',
       'CPC005_A375_6H_X2_B3_DUO52HI53LO:C19'], 
      dtype='|S46')

In [6]:
print np.in1d(PROBES, PROBES_LM1000).sum()
print np.in1d(PROBES, PROBES_LM1000)[:978].sum()

978
978


In [7]:
def slice_matrix(gctx, cids, rids):
    '''Slice the mat by cids and rids and ensure the mat 
    is ordered by cids and rids.'''    
    all_cids = gctx['/0/META/COL/id']
    c_mask = np.in1d(all_cids, cids)
    cids_subset = all_cids[c_mask].tolist()
    c_indices = np.array([cids_subset.index(id_) 
                          for id_ in cids])

    mat = gctx['/0/DATA/0/matrix']
    submat = mat[c_mask, :][c_indices, :]
    
    all_rids = gctx['/0/META/ROW/id']
    r_mask = np.in1d(all_rids, rids)
    rids_subset = all_rids[r_mask].tolist()
    r_indices = np.array([rids_subset.index(id_) 
                          for id_ in rids])
    submat = submat[:, r_mask][:, r_indices]

    return submat

# 1. Connect to MongoDB to retrieve all `sig_id`s that are `trt_cp`

In [8]:
client = MongoClient("mongodb://146.203.54.131:27017")
DB = client['LINCS_L1000_limma']
COLL_SIG = DB['siginfo'] # to get metadata

In [9]:
# Collect pert_ids and batches
cur = COLL_SIG.find({'pert_type':'trt_cp'}, 
                    {'_id':False, 'sig_id':True, 'distil_id':True})
print cur.count()


206339


In [10]:
# Load the metadata for the currently visualized L1000 signatures in L1000FWD
sig_meta_df = pd.read_csv('../data/metadata-full.tsv', sep='\t').set_index('sig_id')
print sig_meta_df.shape
sig_meta_df.head()

(89419, 6)


,cell,dose,pert_id,perturbation,pvalue,time
sig_id,,,,,,
CPC015_MCF7_6H:BRD-A00546892:10.0,MCF7,10.0,BRD-A00546892,-666,0.0143,6
CPC004_VCAP_6H:BRD-A00546892:10.0,VCAP,10.0,BRD-A00546892,BIPERIDEN,0.2056,6
CPC015_ASC_24H:BRD-A00546892:10.0,ASC,10.0,BRD-A00546892,-666,0.2475,24
CPC004_VCAP_24H:BRD-A00546892:10.0,VCAP,10.0,BRD-A00546892,BIPERIDEN,0.3039,24
CPC015_PHH_24H:BRD-A00546892:10.0,PHH,10.0,BRD-A00546892,-666,0.3584,24


In [11]:
print sig_meta_df['pert_id'].nunique()
pert_id_include = set(sig_meta_df['pert_id'].unique())


5312


In [12]:
# Retrieve sig_meta from the MongoDB
cur = COLL_SIG.find({'pert_type':'trt_cp'},
                    {'_id':False, 
                     'sig_id':True, 
                     'distil_id':True,
                    })
sig_meta_df_full = pd.DataFrame([doc for doc in cur]).set_index('sig_id')
print sig_meta_df_full.shape

cur = COLL_SIG.find({'$and': [ 
                        {'pert_type':'trt_cp'},
                        {'distil_nsample': {'$gt': 1}}
                    ]}, 
                    {'_id':False, 
                     'sig_id':True, 
                     'distil_id':True,
                     'pert_id': True,
                     'pert_dose':True,
                     'pert_time':True,
                     'cell_id':True,
                     'pert_desc':True,
                    })
print cur.count()

sig_meta_df = pd.DataFrame([doc for doc in cur if doc['pert_id'] in pert_id_include]).set_index('sig_id')
print sig_meta_df.shape
sig_meta_df.head()

(206339, 1)
202666
(128017, 6)


,cell_id,distil_id,pert_desc,pert_dose,pert_id,pert_time
sig_id,,,,,,
CVD001_HUH7_24H:BRD-K07762753-001-03-6:50,HUH7,"[CVD001_HUH7_24H_X1_F1B3_DUO52HI53LO:M03, CVD0...",-666,50.0,BRD-K07762753,24.0
CPC004_PC3_6H:BRD-K34820100-001-02-1:10,PC3,"[CPC004_PC3_6H_X1_B3_DUO52HI53LO:N23, CPC004_P...",TEBUTHIURON,10.0,BRD-K34820100,6.0
CPC004_PC3_6H:BRD-A22844106-001-16-1:10,PC3,"[CPC004_PC3_6H_X1_B3_DUO52HI53LO:I08, CPC004_P...",TENOXICAM,10.0,BRD-A22844106,6.0
CPC004_PC3_6H:BRD-A55393291-001-05-7:10,PC3,"[CPC004_PC3_6H_X1_B3_DUO52HI53LO:D03, CPC004_P...",TESTOSTERONE,10.0,BRD-A55393291,6.0
CPC004_PC3_6H:BRD-A93255169-001-13-5:10,PC3,"[CPC004_PC3_6H_X1_B3_DUO52HI53LO:F12, CPC004_P...",THALIDOMIDE,10.0,BRD-A93255169,6.0


In [13]:
# Get batch info
sig_meta_df_full['batch'] = sig_meta_df_full.index.map(lambda x:x.split(':')[0])
print sig_meta_df_full['batch'].nunique()

sig_meta_df['batch'] = sig_meta_df.index.map(lambda x:x.split(':')[0])
print sig_meta_df['batch'].nunique()

654
651


In [14]:
# Get all the distil_ids
COLL_INST = DB['instinfo']
cur = COLL_INST.find({'pert_type': {'$in' : ['trt_cp', 'trt_poscon']}}, 
                     {'_id':False, 
                      'distil_id':True, 
                      'pert_id':True,
                      'det_plate':True,
                     })

distil_df = pd.DataFrame([doc for doc in cur]).set_index('distil_id')
print distil_df.shape
distil_df.head()

(675657, 2)


,det_plate,pert_id
distil_id,,
ASG001_MCF7_24H_X1_B7_DUO52HI53LO:A06,ASG001_MCF7_24H_X1_B7_DUO52HI53LO,BRD-A60070924
ASG001_MCF7_24H_X1_B7_DUO52HI53LO:B06,ASG001_MCF7_24H_X1_B7_DUO52HI53LO,BRD-A60070924
ASG001_MCF7_24H_X1_B7_DUO52HI53LO:C06,ASG001_MCF7_24H_X1_B7_DUO52HI53LO,BRD-A60070924
ASG001_MCF7_24H_X1_B7_DUO52HI53LO:D06,ASG001_MCF7_24H_X1_B7_DUO52HI53LO,BRD-A60070924
ASG001_MCF7_24H_X1_B7_DUO52HI53LO:E06,ASG001_MCF7_24H_X1_B7_DUO52HI53LO,BRD-A60070924


In [15]:
def distil_id_to_rna_plate(did):
    return '_'.join(did.split('_')[:4])

def distil_id_to_batch(did):
    return '_'.join(did.split('_')[:3])

def distil_id_to_det_plate(did):
    return '_'.join(did.split('_')[:5])

# distil_df['det_plate'] = distil_df.index.map(distil_id_to_det_plate)
distil_df['rna_plate'] = distil_df.index.map(distil_id_to_rna_plate)
distil_df['batch'] = distil_df.index.map(distil_id_to_batch)
distil_df.head()

,det_plate,pert_id,rna_plate,batch
distil_id,,,,
ASG001_MCF7_24H_X1_B7_DUO52HI53LO:A06,ASG001_MCF7_24H_X1_B7_DUO52HI53LO,BRD-A60070924,ASG001_MCF7_24H_X1,ASG001_MCF7_24H
ASG001_MCF7_24H_X1_B7_DUO52HI53LO:B06,ASG001_MCF7_24H_X1_B7_DUO52HI53LO,BRD-A60070924,ASG001_MCF7_24H_X1,ASG001_MCF7_24H
ASG001_MCF7_24H_X1_B7_DUO52HI53LO:C06,ASG001_MCF7_24H_X1_B7_DUO52HI53LO,BRD-A60070924,ASG001_MCF7_24H_X1,ASG001_MCF7_24H
ASG001_MCF7_24H_X1_B7_DUO52HI53LO:D06,ASG001_MCF7_24H_X1_B7_DUO52HI53LO,BRD-A60070924,ASG001_MCF7_24H_X1,ASG001_MCF7_24H
ASG001_MCF7_24H_X1_B7_DUO52HI53LO:E06,ASG001_MCF7_24H_X1_B7_DUO52HI53LO,BRD-A60070924,ASG001_MCF7_24H_X1,ASG001_MCF7_24H


# 2. Apply normalization for each batch, then compute signatures and insert in to MongoDB

The normalization method is described in [Iskar et al PLoS comp Bio, 2010]():

```
for batch in batches:
    1. discard control samples
    2. mean center each probe across treatment samples in batch
    3. average replicate treatment samples as the signature
```

In [16]:
# Make MongoDB to store those data
# client_local = MongoClient('mongodb://127.0.0.1:27017')
# db = client_local['L1000FWD']
db = client['L1000FWD']
coll = db['sigs']
coll.create_index('sig_id', unique=True)

u'sig_id_1'

In [17]:
from joblib import delayed, Parallel

In [18]:
def compute_signatures(sig_id, row, distil_ids_sub, mat, mat_centered, PROBES_LM1000):
    distil_ids_pert = row['distil_id']
    # Make the sample_class
    mask_pert = np.in1d(distil_ids_sub, distil_ids_pert)
    sample_class = mask_pert.astype(int) + 1
    sample_class = list(sample_class)

    # Apply CD on the original mat
    cd1 = geode.chdir(mat.T, sample_class, PROBES_LM1000, 
                      calculate_sig=0, sort=False, gamma=0.5)

    # Apply CD on the mean centered mat
    cd2 = geode.chdir(mat_centered.T, sample_class, PROBES_LM1000, 
                      calculate_sig=0, sort=False, gamma=0.5)
    # Averaging profiles after mean centering
    avg_vals = mat_centered[mask_pert].mean(axis=0)

    doc = row.to_dict()
    doc['sig_id'] = sig_id
    doc['CD_nocenter_LM'] = list(np.array([item[0] for item in cd1], dtype=np.float64))
    doc['CD_center_LM'] = list(np.array([item[0] for item in cd2], dtype=np.float64))
    doc['avg_center_LM'] = list(avg_vals.astype(np.float64))

    return doc

## The first round of recomputing signatures: `mat` was centered by `batch` like `CPC004_PC3_6H`

1. CD_nocenter_LM
2. CD_center_LM
3. avg_center_LM

In [21]:
# Get all inserted document sig_ids
inserted_sig_ids = set(coll.distinct('sig_id'))
print 'Number of sig_ids inserted: %d' % len(inserted_sig_ids)

sig_ids_left = list(set(sig_meta_df.index) - inserted_sig_ids)

# subset the sig_meta_df
sig_meta_df_left = sig_meta_df.ix[sig_ids_left]
all_batches = sig_meta_df_left['batch'].unique()
n_batches = len(all_batches)
print sig_meta_df_left.shape

for c, batch in enumerate(all_batches):
    sig_meta_df_sub = sig_meta_df_left.query('batch == "%s"' % batch)
    
    # all the treatment samples in this batch
    distil_ids_sub = reduce(lambda x, y: x + y, 
                            sig_meta_df_full.query('batch == "%s"' % batch)['distil_id'])
    print c, n_batches
    print '\t', batch, sig_meta_df_sub.shape, len(distil_ids_sub)
    # Slice the matrix
    mat = slice_matrix(gctx, distil_ids_sub, PROBES_LM1000)
    print '\t', mat.shape
    # Mean center the probes
    mat_centered = mat - mat.mean(axis=0)
    try:
        docs = Parallel(n_jobs=7, backend='multiprocessing', verbose=10)(\
                                  delayed(compute_signatures)(sig_id, row, distil_ids_sub, mat, mat_centered, PROBES_LM1000)\
                                  for sig_id, row in sig_meta_df_sub.iterrows())
    except:
        pass
    else:                     
        result = coll.insert_many(docs)


    


Number of sig_ids inserted: 127963
(54, 7)
0 1
	NMH001_NEU_24H (54, 7) 1023
	(1023, 978)


In [23]:
print sig_meta_df.shape
sig_meta_df.to_csv('data/sig_metadata_%d.csv' % sig_meta_df.shape[0])

(128017, 7)


## The 2nd round of recomputing signatures: `mat` was centered by `det_plate` like `ASG001_MCF7_24H_X1_B7`

In [19]:
def mean_center(mat, centerby):
    '''Mean center a mat based on centerby. mat is a samples x genes matrix'''
    mat_centered = np.zeros_like(mat)
    
    for group in set(centerby):
        mask = np.in1d(centerby, [group])
        mat_centered[mask] = mat[mask] - mat[mask].mean(axis=0)
    
    return mat_centered

In [21]:
def compute_signatures2(sig_id, row, distil_ids_sub, mat_centered, PROBES_LM1000):
    distil_ids_pert = row['distil_id']
    # Make the sample_class
    mask_pert = np.in1d(distil_ids_sub, distil_ids_pert)
    sample_class = mask_pert.astype(int) + 1

    # Apply CD on the mean centered mat
    cd = geode.chdir(mat_centered.T, sample_class, PROBES_LM1000, 
                      calculate_sig=0, sort=False, gamma=0.5)
    # Averaging profiles after mean centering
    avg_vals = mat_centered[mask_pert].mean(axis=0)

    doc = {}
    doc['sig_id'] = sig_id
    doc['CD_center_LM_det'] = list(np.array([item[0] for item in cd], dtype=np.float64))
    doc['avg_center_LM_det'] = list(avg_vals.astype(np.float64))

    return doc

In [22]:
def compute_sig2_wrapper(sig_id, row, distil_ids_sub, mat_centered, PROBES_LM1000):
    try:
        doc = compute_signatures2(sig_id, row, distil_ids_sub, mat_centered, PROBES_LM1000)
    except ValueError as e:
        doc = None
        print e
        pass
    return doc

In [23]:
key = 'CD_center_LM_det'
# Get all inserted document sig_ids
inserted_sig_ids = set(coll.find({key: {'$exists': True}}).distinct('sig_id'))
print 'Number of sig_ids inserted: %d' % len(inserted_sig_ids)


Number of sig_ids inserted: 27429


In [ ]:
sig_ids_left = list(set(sig_meta_df.index) - inserted_sig_ids)

# subset the sig_meta_df
sig_meta_df_left = sig_meta_df.ix[sig_ids_left]
all_batches = sig_meta_df_left['batch'].unique()
n_batches = len(all_batches)
print sig_meta_df_left.shape

for c, batch in enumerate(all_batches):
    sig_meta_df_sub = sig_meta_df_left.query('batch == "%s"' % batch)
    
    # all the treatment samples in this batch
    distil_ids_sub_df = distil_df.query('batch == "%s"' % batch)
    distil_ids_sub = distil_ids_sub_df.index.tolist()
    
    print c, n_batches
    print '\t', batch, sig_meta_df_sub.shape, len(distil_ids_sub)
    # Slice the matrix
    mat = slice_matrix(gctx, distil_ids_sub, PROBES_LM1000)
    print '\t', mat.shape
    # Mean center the probes by det_plate
    mat_centered = mean_center(mat, distil_ids_sub_df['det_plate'])
    
    try:
        docs = Parallel(n_jobs=7, backend='multiprocessing', verbose=10)(\
                                      delayed(compute_sig2_wrapper)(sig_id, row, distil_ids_sub, mat_centered, PROBES_LM1000)\
                                      for sig_id, row in sig_meta_df_sub.iterrows())
        docs = filter(None, docs)
    except Exception as e:
        print e
        pass
    else:
        if len(docs) > 0:
            bulk = coll.initialize_ordered_bulk_op()
            for doc in docs:
                bulk.find({'sig_id': doc['sig_id']}).\
                    update_one({'$set': {
                        'CD_center_LM_det': doc['CD_center_LM_det'],
                        'avg_center_LM_det': doc['avg_center_LM_det']
                    }})
            bulk.execute()    


(100588, 7)
0 497
	HOG001_A549_24H (277, 7) 858
	(858, 978)


[Parallel(n_jobs=7)]: Done   1 out of 277 | elapsed:    0.9s remaining:  4.2min
[Parallel(n_jobs=7)]: Done  27 out of 277 | elapsed:    4.3s remaining:   40.0s
[Parallel(n_jobs=7)]: Done  55 out of 277 | elapsed:    8.6s remaining:   34.7s
[Parallel(n_jobs=7)]: Done  83 out of 277 | elapsed:   13.3s remaining:   31.1s
[Parallel(n_jobs=7)]: Done 111 out of 277 | elapsed:   17.7s remaining:   26.5s
[Parallel(n_jobs=7)]: Done 139 out of 277 | elapsed:   22.4s remaining:   22.3s
[Parallel(n_jobs=7)]: Done 167 out of 277 | elapsed:   27.5s remaining:   18.1s
[Parallel(n_jobs=7)]: Done 195 out of 277 | elapsed:   32.5s remaining:   13.7s
[Parallel(n_jobs=7)]: Done 223 out of 277 | elapsed:   37.3s remaining:    9.0s
[Parallel(n_jobs=7)]: Done 251 out of 277 | elapsed:   41.9s remaining:    4.3s
[Parallel(n_jobs=7)]: Done 277 out of 277 | elapsed:   46.2s finished


1 497
	PCLB003_A549_24H (176, 7) 1094
	(1094, 978)


[Parallel(n_jobs=7)]: Done   1 out of 176 | elapsed:    1.5s remaining:  4.4min
[Parallel(n_jobs=7)]: Done  16 out of 176 | elapsed:    4.8s remaining:   47.9s
[Parallel(n_jobs=7)]: Done  34 out of 176 | elapsed:    9.1s remaining:   38.0s
[Parallel(n_jobs=7)]: Done  52 out of 176 | elapsed:   13.7s remaining:   32.6s
[Parallel(n_jobs=7)]: Done  70 out of 176 | elapsed:   18.8s remaining:   28.5s
[Parallel(n_jobs=7)]: Done  88 out of 176 | elapsed:   23.4s remaining:   23.4s
[Parallel(n_jobs=7)]: Done 106 out of 176 | elapsed:   27.9s remaining:   18.4s
[Parallel(n_jobs=7)]: Done 124 out of 176 | elapsed:   32.3s remaining:   13.6s
[Parallel(n_jobs=7)]: Done 142 out of 176 | elapsed:   37.7s remaining:    9.0s
[Parallel(n_jobs=7)]: Done 160 out of 176 | elapsed:   42.1s remaining:    4.2s
[Parallel(n_jobs=7)]: Done 176 out of 176 | elapsed:   45.9s finished


2 497
	HOG002_MCF7_24H (287, 7) 852
	(852, 978)


...

[Parallel(n_jobs=7)]: Done   1 out of 353 | elapsed:    1.8s remaining: 10.4min
[Parallel(n_jobs=7)]: Done  31 out of 353 | elapsed:   11.7s remaining:  2.0min
[Parallel(n_jobs=7)]: Done  67 out of 353 | elapsed:   23.8s remaining:  1.7min
[Parallel(n_jobs=7)]: Done 103 out of 353 | elapsed:   35.8s remaining:  1.4min
[Parallel(n_jobs=7)]: Done 139 out of 353 | elapsed:   48.0s remaining:  1.2min
[Parallel(n_jobs=7)]: Done 175 out of 353 | elapsed:  1.0min remaining:  1.0min
[Parallel(n_jobs=7)]: Done 211 out of 353 | elapsed:  1.2min remaining:   48.7s
[Parallel(n_jobs=7)]: Done 247 out of 353 | elapsed:  1.4min remaining:   36.4s
[Parallel(n_jobs=7)]: Done 283 out of 353 | elapsed:  1.6min remaining:   23.9s
[Parallel(n_jobs=7)]: Done 319 out of 353 | elapsed:  1.8min remaining:   11.6s
[Parallel(n_jobs=7)]: Done 353 out of 353 | elapsed:  2.0min finished


53 497
	CPC002_PC3_24H (363, 7) 1781
	(1781, 978)


[Parallel(n_jobs=7)]: Done   1 out of 363 | elapsed:    3.8s remaining: 23.0min
[Parallel(n_jobs=7)]: Done  32 out of 363 | elapsed:   22.4s remaining:  3.9min
[Parallel(n_jobs=7)]: Done  69 out of 363 | elapsed:   45.4s remaining:  3.2min
[Parallel(n_jobs=7)]: Done 106 out of 363 | elapsed:  1.2min remaining:  2.9min
[Parallel(n_jobs=7)]: Done 143 out of 363 | elapsed:  1.6min remaining:  2.4min
[Parallel(n_jobs=7)]: Done 180 out of 363 | elapsed:  2.0min remaining:  2.0min
[Parallel(n_jobs=7)]: Done 217 out of 363 | elapsed:  2.4min remaining:  1.6min
[Parallel(n_jobs=7)]: Done 254 out of 363 | elapsed:  2.8min remaining:  1.2min
[Parallel(n_jobs=7)]: Done 291 out of 363 | elapsed:  3.2min remaining:   47.7s
[Parallel(n_jobs=7)]: Done 328 out of 363 | elapsed:  3.6min remaining:   23.2s
[Parallel(n_jobs=7)]: Done 363 out of 363 | elapsed:  4.0min finished


54 497
	CPC015_MCF7_24H (361, 7) 1772
	(1772, 978)


[Parallel(n_jobs=7)]: Done   1 out of 361 | elapsed:    3.8s remaining: 23.1min
[Parallel(n_jobs=7)]: Done  30 out of 361 | elapsed:   23.4s remaining:  4.3min
[Parallel(n_jobs=7)]: Done  67 out of 361 | elapsed:   46.9s remaining:  3.4min
[Parallel(n_jobs=7)]: Done 104 out of 361 | elapsed:  1.2min remaining:  3.0min


In [58]:
def chdir_avg(mat, sample_class, batches, PROBES_LM1000):
    '''Compute CD on each batch, then average them.
    '''
    n_batches = len(set(batches))
    cds_all = []
    for batch in set(batches):
        mask_batch = np.in1d(batches, [batch])
        if len(set(sample_class[mask_batch])) >= 1:
        
            cd = geode.chdir(mat[:, mask_batch], sample_class[mask_batch], PROBES_LM1000, 
                              calculate_sig=0, sort=False, gamma=0.5)
            cd_coefs = np.array([item[0] for item in cd])
            cds_all.append(cd_coefs)

    return np.array(cds_all).mean(axis=0)


In [59]:
def compute_signatures3(sig_id, row, distil_ids_sub_df, mat_centered, PROBES_LM1000):
    distil_ids_pert = row['distil_id']
    # Make the sample_class
    distil_ids_sub = distil_ids_sub_df.index.tolist()
    
    mask_pert = np.in1d(distil_ids_sub, distil_ids_pert)
    sample_class = mask_pert.astype(int) + 1
    
    batches = distil_ids_sub_df['det_plate']

    # Apply CD on the mean centered mat
    cd = chdir_avg(mat_centered.T, sample_class, batches, PROBES_LM1000)

    doc = {}
    doc['sig_id'] = sig_id
    doc['CDavg_center_LM_det'] = list(cd.astype(np.float64))

    return doc

In [60]:
# Get all inserted document sig_ids
key = 'CDavg_center_LM_det'
inserted_sig_ids = set(coll.find({key: {'$exists': True}}).distinct('sig_id'))
print 'Number of sig_ids inserted: %d' % len(inserted_sig_ids)

sig_ids_left = list(set(sig_meta_df.index) - inserted_sig_ids)

# subset the sig_meta_df
sig_meta_df_left = sig_meta_df.ix[sig_ids_left]
all_batches = sig_meta_df_left['batch'].unique()
n_batches = len(all_batches)
print sig_meta_df_left.shape

for c, batch in enumerate(all_batches):
    sig_meta_df_sub = sig_meta_df_left.query('batch == "%s"' % batch)
    
    # all the treatment samples in this batch
    distil_ids_sub_df = distil_df.query('batch == "%s"' % batch)
    distil_ids_sub = distil_ids_sub_df.index.tolist()
    
    print c, n_batches
    print '\t', batch, sig_meta_df_sub.shape, len(distil_ids_sub)
    # Slice the matrix
    mat = slice_matrix(gctx, distil_ids_sub, PROBES_LM1000)
    print '\t', mat.shape
    # Mean center the probes by det_plate
    mat_centered = mean_center(mat, distil_ids_sub_df['det_plate'])
    
    for sig_id, row in sig_meta_df_sub.iterrows():
        try:
            cd = compute_signatures3(sig_id, row, distil_ids_sub_df, mat_centered, PROBES_LM1000)
        except Exception as e:
            print e
            break

#     try:
#         docs = Parallel(n_jobs=10, backend='multiprocessing', verbose=10)(\
#                                       delayed(compute_signatures3)(sig_id, row, distil_ids_sub_df, mat_centered, PROBES_LM1000)\
#                                       for sig_id, row in sig_meta_df_sub.iterrows())
#     except Exception as e:
#         print e
#         pass
#     else:                     
#         bulk = coll.initialize_ordered_bulk_op()
#         for doc in docs:
#             bulk.find({'sig_id': doc['sig_id']}).\
#                 update_one({'$set': {
#                     'CDavg_center_LM_det': doc['CDavg_center_LM_det']
#                 }})
#         bulk.execute()    
    break

Number of sig_ids inserted: 0
(128017, 7)
0 651
	LJP001_HS578T_24H (310, 7) 1012
	(1012, 978)
sampleclass has to be a list whose elements are in only 0, 1 or 2


In [43]:
distil_ids_sub_df.shape

(1012, 4)

In [45]:
distil_ids_pert = row['distil_id']
# Make the sample_class
distil_ids_sub = distil_ids_sub_df.index.tolist()

mask_pert = np.in1d(distil_ids_sub, distil_ids_pert)
sample_class = mask_pert.astype(int) + 1

batches = distil_ids_sub_df['det_plate']

# Apply CD on the mean centered mat
cd = chdir_avg(mat_centered.T, sample_class, batches, PROBES_LM1000)


ValueError: sampleclass has to be a list whose elements are in only 0, 1 or 2

In [46]:
print sample_class.shape, batches.shape

(1012,) (1012,)


In [53]:
for batch in set(batches):
    mask_batch = np.in1d(batches, [batch])
    print mat_centered[mask_batch].shape, sample_class[mask_batch]
#     cd = geode.chdir(mat_centered[:, mask_batch], sample_class[mask_batch], PROBES_LM1000, 
#                       calculate_sig=0, sort=False, gamma=0.5)


(338, 978) [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1]
(337, 978) [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1

In [57]:
1>=2

False

In [39]:
cd = chdir_avg(mat_centered.T, sample_class, batches, PROBES_LM1000)

In [40]:
cd.shape

(978,)